# Step 0: Create spark context

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
spark

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Step 1: Get data with which we are familiar



### Dataset: Green practices across DC
[documentation](https://opendata.dc.gov/datasets/best-management-practices/data)
![rainbarrel](https://cdn11.bigcommerce.com/s-j602wc6a/products/7096/images/21615/great-american-rainbarrel-trio__96479.1502809069.500.750.jpg?c=2)

We want to know:
- what are the most common BMP_Type by neighborhood? 
- How many instances are on private vs public land?

### Step 2: Create a pandas dataframe

In [10]:
import pandas as pd
pandaDf = pd.read_csv('https://opendata.arcgis.com/datasets/a973c2c7b7c14a918859f3e38bdffdd2_42.csv')

/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
pandaDf.head()

,X,Y,BMP_ID_NUMBER,BMP_TYPE,BMP_SUB_TYPE,UNDERDRAIN,BMP_GROUP,INSTALLATION_DATE,DATE_REMOVED,CONTRIBUTING_DRAINAGE_AREA_FT2,...,BMP_NUMBER,PLAN_NUMBER,BMP_TYPE_ACCESS,DATE_APPROVED,SEWERSHED,INCLUDE_BUILT,INCLUDE_ALL,WARD,GREEN_ROOF_REBATE,RIVERSMART_HOMES
0,-77.036773,38.958677,R15760-1-2,Shade Tree,NaN,False,Tree Planting and Preservation,2014-05-14T00:00:00.000Z,NaN,0,...,#R15760-1-2,NaN,NaN,NaN,MS4,True,True,4.0,No,Yes
1,-76.938687,38.888273,R11333-1-12,Shade Tree,NaN,False,Tree Planting and Preservation,2017-03-20T00:00:00.000Z,NaN,0,...,#R11333-1-12,NaN,NaN,NaN,MS4,True,True,7.0,No,Yes
2,-76.989573,38.928465,R12966-1-2,Rain Barrel,NaN,False,Rainwater Harvesting,2012-06-07T00:00:00.000Z,NaN,220,...,#R12966-1-2,NaN,NaN,NaN,CSS,True,True,5.0,No,Yes
3,-77.031645,38.954937,R13157-1-6,Shade Tree,NaN,False,Tree Planting and Preservation,2012-09-01T00:00:00.000Z,NaN,0,...,#R13157-1-6,NaN,NaN,NaN,CSS,True,True,4.0,No,Yes
4,-77.007900,38.827600,4719-1-5,Infiltration trench,NaN,False,Infiltration,2018-06-21T00:00:00.000Z,NaN,9,...,#4719-1-5,4719,NaN,2016-06-21T00:00:00.000Z,MS4,True,True,8.0,No,No


In [16]:
import pandas_profiling as pp



In [17]:
pp.ProfileReport(pandaDf)

Number of variables,51
Number of observations,19975
Total Missing (%),26.6%
Total size in memory,7.2 MiB
Average record size in memory,380.0 B
Numeric,16
Categorical,22
Boolean,2
Date,0
Text (Unique),0
Rejected,10


In [23]:
pd.crosstab(pandaDf['WARD'], pandaDf['BMP_TYPE'])

BMP_TYPE,Bayscaping,Bioretention,CDA to a Shared BMP,Dry swale,Engineered tree pits,Engineered treepits,Extensive green roof,Filtering System,Filtering Systems,Grass channel,...,Streetscape bioretention,Surface sand filter,Three chamber underground sand filter,Traditional bioretention,Tree planting,Tree preservation,Underground vault,Wet Swale,Wet pond,Wetlands
WARD,,,,,,,,,,,,,,,,,,,,,
1.0,89,26,0,1,0,1,32,2,48,0,...,2,0,0,30,18,6,0,3,0,0
2.0,8,34,0,2,0,6,180,14,105,1,...,14,0,0,91,18,8,2,0,0,0
3.0,153,91,2,4,0,5,28,8,49,12,...,0,0,0,16,111,53,0,0,0,1
4.0,393,139,0,16,0,2,21,1,22,1,...,5,0,0,29,141,20,0,0,0,0
5.0,307,161,1,18,0,3,49,3,54,4,...,1,1,0,70,218,10,0,0,0,2
6.0,102,147,1,16,2,117,93,8,102,8,...,42,5,1,64,91,23,5,2,0,1
7.0,229,95,0,10,0,2,7,1,54,10,...,1,0,0,70,39,4,0,0,0,0
8.0,108,113,1,14,0,0,14,3,32,6,...,1,0,3,174,67,11,2,0,1,0


### Step 3: Create spark dataframe

In [32]:
myrdd = sc.textFile('https://opendata.arcgis.com/datasets/a973c2c7b7c14a918859f3e38bdffdd2_42.csv').map(lambda line: line.split(","))
# need to make this a df


In [37]:
" ".join(pandaDf.columns)

'X Y BMP_ID_NUMBER BMP_TYPE BMP_SUB_TYPE UNDERDRAIN BMP_GROUP INSTALLATION_DATE DATE_REMOVED CONTRIBUTING_DRAINAGE_AREA_FT2 POST_PROJECT_NATURAL_FT2 POST_PROJECT_COMPACTED_FT2 POST_PROJECT_IMPERVIOUS_FT2 POST_PROJECT_BMP_AREA_FT2 POST_PROJECT_VEHICULAR_FT2 PRE_PROJECT_NATURAL_FT2 PRE_PROJECT_COMPACTED_FT2 PRE_PROJECT_IMPERVIOUS_FT2 PRE_PROJECT_BMP_AREA_FT2 PRE_PROJECT_VEHICULAR_FT2 STORAGE_VOLUME_FT3 RETENTION_VOLUME_FT3 ADDITIONAL_VOLUME_TREATED_FT3 X_COORDINATE Y_COORDINATE MAJOR_DRAINAGE_BASIN MINOR_DRAINAGE_BASIN NUMBER_OF_TREES PROPRIETARY_PRACTICE_NAME DESCRIBE_PROPRIETARY_PRACTICE PROJECT_TYPE GIS_LAST_MOD_DTTM LATITUDE LONGITUDE OBJECTID BMP_GROUP_ABBREVIATION MAJOR_REGULATED_ACTIVITY SRC_GENERATION RIVERSMART_REWARDS GRAY_OR_GREEN_INFRASTRUCTURE PUBLIC_RIGHT_OF_WAY BMP_NUMBER PLAN_NUMBER BMP_TYPE_ACCESS DATE_APPROVED SEWERSHED INCLUDE_BUILT INCLUDE_ALL WARD GREEN_ROOF_REBATE RIVERSMART_HOMES'

In [39]:
schemaString = " ".join(pandaDf.columns)
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

spark.createDataFrame(myrdd)


NameError: name 'StructField' is not defined

In [53]:
type(pandaDf)

pandas.core.frame.DataFrame

In [52]:
type(df)

pyspark.sql.dataframe.DataFrame

In [54]:
df = spark.read.csv('../../data/Best_Management_Practices.csv', sep=',', comment=None, header=True, inferSchema=True, mode="DROPMALFORMED")
df.head()

Row(X=-77.03677300762558, Y=38.958676906847835, BMP_ID_NUMBER='R15760-1-2', BMP_TYPE='Shade Tree', BMP_SUB_TYPE=None, UNDERDRAIN=False, BMP_GROUP='Tree Planting and Preservation', INSTALLATION_DATE=datetime.datetime(2014, 5, 13, 20, 0), DATE_REMOVED=None, CONTRIBUTING_DRAINAGE_AREA_FT2=0, POST_PROJECT_NATURAL_FT2=None, POST_PROJECT_COMPACTED_FT2=None, POST_PROJECT_IMPERVIOUS_FT2=0.0, POST_PROJECT_BMP_AREA_FT2=None, POST_PROJECT_VEHICULAR_FT2=None, PRE_PROJECT_NATURAL_FT2=None, PRE_PROJECT_COMPACTED_FT2=None, PRE_PROJECT_IMPERVIOUS_FT2=None, PRE_PROJECT_BMP_AREA_FT2=None, PRE_PROJECT_VEHICULAR_FT2=None, STORAGE_VOLUME_FT3=None, RETENTION_VOLUME_FT3=10, ADDITIONAL_VOLUME_TREATED_FT3=0.0, X_COORDINATE=396812.99, Y_COORDINATE=143418.72, MAJOR_DRAINAGE_BASIN='Rock Creek', MINOR_DRAINAGE_BASIN='Rock Creek', NUMBER_OF_TREES=1, PROPRIETARY_PRACTICE_NAME=None, DESCRIBE_PROPRIETARY_PRACTICE=None, PROJECT_TYPE=None, GIS_LAST_MOD_DTTM=datetime.datetime(2019, 6, 17, 2, 0, 18), LATITUDE=38.958669108

In [43]:
df.persist()

DataFrame[X: double, Y: double, BMP_ID_NUMBER: string, BMP_TYPE: string, BMP_SUB_TYPE: string, UNDERDRAIN: boolean, BMP_GROUP: string, INSTALLATION_DATE: timestamp, DATE_REMOVED: timestamp, CONTRIBUTING_DRAINAGE_AREA_FT2: int, POST_PROJECT_NATURAL_FT2: double, POST_PROJECT_COMPACTED_FT2: double, POST_PROJECT_IMPERVIOUS_FT2: double, POST_PROJECT_BMP_AREA_FT2: int, POST_PROJECT_VEHICULAR_FT2: double, PRE_PROJECT_NATURAL_FT2: int, PRE_PROJECT_COMPACTED_FT2: double, PRE_PROJECT_IMPERVIOUS_FT2: double, PRE_PROJECT_BMP_AREA_FT2: double, PRE_PROJECT_VEHICULAR_FT2: double, STORAGE_VOLUME_FT3: double, RETENTION_VOLUME_FT3: int, ADDITIONAL_VOLUME_TREATED_FT3: double, X_COORDINATE: double, Y_COORDINATE: double, MAJOR_DRAINAGE_BASIN: string, MINOR_DRAINAGE_BASIN: string, NUMBER_OF_TREES: int, PROPRIETARY_PRACTICE_NAME: string, DESCRIBE_PROPRIETARY_PRACTICE: string, PROJECT_TYPE: string, GIS_LAST_MOD_DTTM: timestamp, LATITUDE: double, LONGITUDE: double, OBJECTID: int, BMP_GROUP_ABBREVIATION: string

In [44]:
df.limit(5).toPandas()
#df.head()
#df.show(5)

,X,Y,BMP_ID_NUMBER,BMP_TYPE,BMP_SUB_TYPE,UNDERDRAIN,BMP_GROUP,INSTALLATION_DATE,DATE_REMOVED,CONTRIBUTING_DRAINAGE_AREA_FT2,...,BMP_NUMBER,PLAN_NUMBER,BMP_TYPE_ACCESS,DATE_APPROVED,SEWERSHED,INCLUDE_BUILT,INCLUDE_ALL,WARD,GREEN_ROOF_REBATE,RIVERSMART_HOMES
0,-77.036773,38.958677,R15760-1-2,Shade Tree,None,False,Tree Planting and Preservation,2014-05-13 20:00:00,None,0,...,#R15760-1-2,None,None,NaT,MS4,True,True,4,No,Yes
1,-76.938687,38.888273,R11333-1-12,Shade Tree,None,False,Tree Planting and Preservation,2017-03-19 20:00:00,None,0,...,#R11333-1-12,None,None,NaT,MS4,True,True,7,No,Yes
2,-76.989573,38.928465,R12966-1-2,Rain Barrel,None,False,Rainwater Harvesting,2012-06-06 20:00:00,None,220,...,#R12966-1-2,None,None,NaT,CSS,True,True,5,No,Yes
3,-77.031645,38.954937,R13157-1-6,Shade Tree,None,False,Tree Planting and Preservation,2012-08-31 20:00:00,None,0,...,#R13157-1-6,None,None,NaT,CSS,True,True,4,No,Yes
4,-77.007900,38.827600,4719-1-5,Infiltration trench,None,False,Infiltration,2018-06-20 20:00:00,None,9,...,#4719-1-5,4719,None,2016-06-20 20:00:00,MS4,True,True,8,No,No


In [50]:
# Select columns using DF format


In [47]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [48]:
# Change code to relevant names and variables

import pyspark.sql.functions as F
counts = spark_df.agg(F.countDistinct('BPM_Type'))
query = """
SELECT overall, COUNT(*)
FROM reviews
GROUP BY overall
ORDER BY overall
"""
reviews_df.createOrReplaceTempView('reviews')
output = spark.sql(query)
show(output, n=1000)

NameError: name 'spark_df' is not defined

## Step 4: Practice with some more complex data
Documentation for data can be found [here](http://jmcauley.ucsd.edu/data/amazon/)

In [ ]:
# Download data (run this only once)
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
#!gunzip reviews_Toys_and_Games_5.json.gz